In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#Didn't use any sklearn
#from sklearn.model_selection import train_test_split
#from sklearn.model_selection import KFold
#from sklearn import preprocessing
#from sklearn.neural_network import MLPClassifier
from random import randrange
from random import seed
from random import randrange
from random import random
from math import exp
from csv import reader

#X = dataset[:,1:14]
#Y = dataset[:,0:1]
#dataset = np.loadtxt('wine.csv',delimiter = ",")


# MLP
# Load wine CSV file
def load_csv():
    dataset = []
    with open('wine.csv', 'r') as file:
        csv_reader = reader(file)
        for i in csv_reader:
            dataset.append(i)
    return dataset


# Conversion (string to float) 
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())


# Conversion (string to int) 
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup


# Finding min/max
def dataset_minmax(dataset):
    minmax = list()
    stats = [[min(column), max(column)] for column in zip(*dataset)]
    return stats


# Rescale dataset
def normalize_dataset(dataset, minmax):
    for row in dataset:
        for i in range(len(row) - 1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])


# Splitting dataset into the (k) folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split


# Accuracy percentage 
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0


# Cross validation split (evaluation)
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores


# Per input, calculatiion of neuron activation
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights) - 1):
        activation += weights[i] * inputs[i]
    return activation


# Neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))


# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs


# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)


# Backpropagation
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network) - 1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])


# Update weights w/ error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']


# Train the network epochs time(s)
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)


# Initialize the network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights': [random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights': [random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network


# Make a prediction from the network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))


# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return (predictions)



#Main class method
if __name__ == '__main__':
    seed(1)
    dataset_other = load_csv()
    new_data = list()
    dataset = []

    for i in dataset_other:
        new_data = i[1:]
        new_data.append(i[0])
        dataset.append(new_data)

    for i in range(len(dataset[0]) - 1):
        str_column_to_float(dataset, i)

    # conversion string class to integers
    str_column_to_int(dataset, len(dataset[0]) - 1)
    minmax = dataset_minmax(dataset)
    normalize_dataset(dataset, minmax)

    # evaluate algorithm
    n_folds = 5
    l_rate = 0.3
    n_epoch = 500
    n_hidden = 5
    scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
    
    print("K-Folds: ", n_folds)
    print("Learning rate: ", l_rate)
    print("Trained for ",n_epoch,"epochs")
    print("Number of nodes in hidden layer: ", n_hidden)
    print('Scores: %s' % scores)
    print('Accuracy: %.3f%%' % (sum(scores) / float(len(scores))))



def load_csv():
    dataset = []
    with open('/Users/nickfrasco/PycharmProjects/Wine_set/wine.csv', 'r') as file:
        csv_reader = reader(file)
        for i in csv_reader:
            dataset.append(i)
    return dataset


print("-------------------")





#<-- Unused -->
#load dataset
#dataset = np.loadtxt('wine.csv',delimiter = ",")
#kf = KFold(n_splits=5)
#kf.get_n_splits(X)
#print(kf)





K-Folds:  5
Learning rate:  0.3
Trained for  500 epochs
Number of nodes in hidden layer:  5
Scores: [97.14285714285714, 100.0, 100.0, 97.14285714285714, 97.14285714285714]
Accuracy: 98.286%
-------------------


## Final Report:
The first thing I did after inputting the data, was make the network. This creates a new neural network, with weights and hidden neurons, that is ready for training data. Each neuron in the output layer connects to each neuron in the hidden layer. I also normalized it, to make it in the range of zero and one (using sigmoid function.) Next I made the forward propagation function; which is comprised of neuron activation, transfer, and then the actual forward propagation. This tests if each neuron should be activated, transfers the output to an action, and then adds all of the outputs from one layer to become inputs to the next. Then comes the calculation of back propagation error, which trains weights. Next comes the training of the network; which is comprised of updating each weight from the errors calculated from the back propagation method, as well as updating the network. Lastly, we predict with the trained neural network. We got a pretty good accuracy rate of 98%!

used for help:
1. www.machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python 
2. https://towardsdatascience.com/perceptron-learning-algorithm-d5db0deab975
3. https://analyticsindiamag.com/a-beginners-guide-to-scikit-learns-mlpclassifier/